In [1]:
from collections import OrderedDict
import numpy as np

def generateTestData(numberOfStartups=11, numberOfInvestors=35, numberOfRanksInvestorCanProvide=2):
    startups = []
    investors = []
    np.random.seed(123448)
    
    for startup in range(numberOfStartups):
        startups.append({
            'name': ("Startup %d" % startup)
        })
    
    
    for investor in range(numberOfInvestors):
        rankedStartups = np.random.choice(startups, numberOfRanksInvestorCanProvide, replace=False)
        investors.append({
            'name': ("Investor %d" % investor),
            'rankedStartups': rankedStartups
        })
    
    
    return (startups, investors)

'''
This function is a good one to adjust/tweak based on how tight/loose we want our 'constraints'
to be -- when super tightly constrained, the probability of there existing
an 'optimal' solution decreases. Conversely,loosening the constraints increases our odds
of finding a solution that works.
'''
def cantBeScheduledTogether(startup1, startup2, investors):
    for investor in investors:
        if (
            startup1['name'] != startup2['name'] and
            (startup1 in investor['rankedStartups'] and
            startup2 in investor['rankedStartups'])
            
        ):
            return True
    return False

'''
this essentially creates our graph:
- nodes: startups
- edges: drawn between startups that we wouldn't want doing
         their Q&As at the same time (based on investor preferences)
'''
def generateStartupGraph(startups, investors):
    startupGraph = OrderedDict()
    
    for startup in startups:
        neighbors = list(filter(lambda x: cantBeScheduledTogether(x, startup, investors), startups))
        neighborNames = list(map(lambda x: x['name'], neighbors))
        startupGraph[startup['name']] = neighborNames
    
    return startupGraph


In [2]:
class CSP:
    def __init__(self, v, neighbors, domain):
        self.variables = v
        self.neighbors = neighbors
        self.domain = domain

class backtrackCount:
    def __init__(self):
        self.count = 0

def backtracking_search(csp):
    assignment = {}
    c = backtrackCount()
    return recursive_backtracking(assignment, csp, c)

def conflictExists(neighbors, assignment, timeSlot):
    for startup in neighbors:
        if startup in assignment.keys():
            if (
                assignment[startup] == timeSlot
            ):
                return True

    return False

def recursive_backtracking(assignment, csp, backtracks):
    if (len(csp.variables) == 0):
        return (OrderedDict(sorted(assignment.items())), backtracks.count)
    
    if (len(assignment.keys()) == 0):
        currentState = csp.variables[len(csp.variables) - 1]
        remainder = csp.variables[:-1]
    else:
        currentState = csp.variables[0]
        remainder = csp.variables[1:]
    
    
    for timeSlot in sorted(csp.domain):
        if (conflictExists(csp.neighbors[currentState['name']], assignment, timeSlot) == False):
            assignment[currentState['name']] = timeSlot
            
            ans = recursive_backtracking(
                assignment,
                CSP(remainder,
                    csp.neighbors,
                    csp.domain
                ),
                backtracks
            )
            
            if ans != None:
                return ans

            backtracks.count += 1
            del assignment[currentState['name']]
    
    return None

def generateSchedule(
    startups, startupGraph,
    idealTimeSlots=1, hoursAvailable=2, minimumSlotDurationMinutes=15,
    metrics=False
):
    timeSlots = [x for x in range(idealTimeSlots)]
    
    slotDuration = float(hoursAvailable*60)/idealTimeSlots
    if (slotDuration < minimumSlotDurationMinutes):
        print("Can't be solved ¯\\_(ツ)_/¯\n")
        return None
    
    csp = CSP(startups, startupGraph, timeSlots)
    a = backtracking_search(csp)
    
    if (a == None):
        return generateSchedule(
            startups, startupGraph,
            (idealTimeSlots + 1), hoursAvailable, minimumSlotDurationMinutes, metrics
        )
    else:
        print('Minimum number of time slots required: %d' % idealTimeSlots)
        print('Number of backtracks: %d\n' % a[1])
        if (metrics == False):
            print('====== Schedule ======')
            for slot in timeSlots:
                print('* Slot %d (%d minutes)' % ((slot+1), slotDuration))
                startupsInSlot = [k for k,v in a[0].items() if v == slot]
                for startup in startupsInSlot:
                    print('  * %s' % startup)

## Example Schedule Generated

In [11]:
'''          
the # of startups/investors we're defaulting to correspond to the
actual attendance of the event this is based on
'''
(startups, investors) = generateTestData(
    numberOfStartups=11,
    numberOfInvestors=35,
    numberOfRanksInvestorCanProvide=2
)

'''
this is currently taking in our made-up data,
but can take any real-life data that is structured as prescribed,
and generate the graph on which we'll conduct recursive backtracking.
'''
startupGraph = generateStartupGraph(startups, investors)

generateSchedule(startups, startupGraph)

Minimum number of time slots required: 4
Number of backtracks: 5

====== Schedule ======
* Slot 1 (30 minutes)
  * Startup 1
  * Startup 10
  * Startup 7
  * Startup 8
* Slot 2 (30 minutes)
  * Startup 0
  * Startup 3
  * Startup 9
* Slot 3 (30 minutes)
  * Startup 2
  * Startup 4
  * Startup 5
* Slot 4 (30 minutes)
  * Startup 6


## Experiment 1: Ratio

In [3]:
ratiosToTest = [
    [11, 35, 2], # the ratio at the actual event
    [11, 60, 2], # seeing if only scaling the investors impacts anything
    [11, 85, 2],
    [11, 110, 2],
    [11, 200, 2],
    [20, 35, 2], # seeing if only scaling the startups impacts anything
    [30, 35, 2],
    [40, 35, 2],
    [11, 35, 5] # seeing if only scaling the # of startups the investors can rank
]

for ratio in ratiosToTest:
    print(ratio)
    (startups, investors) = generateTestData(
        numberOfStartups=ratio[0],
        numberOfInvestors=ratio[1],
        numberOfRanksInvestorCanProvide=ratio[2]
    )
    startupGraph = generateStartupGraph(startups, investors)
    generateSchedule(startups, startupGraph, metrics=True)

[11, 35, 2]
Minimum number of time slots required: 4
Number of backtracks: 5

[11, 60, 2]
Minimum number of time slots required: 7
Number of backtracks: 0

[11, 85, 2]
Minimum number of time slots required: 7
Number of backtracks: 0

[11, 110, 2]
Minimum number of time slots required: 8
Number of backtracks: 0

[11, 200, 2]
Can't be solved ¯\_(ツ)_/¯

[20, 35, 2]
Minimum number of time slots required: 3
Number of backtracks: 171

[30, 35, 2]
Minimum number of time slots required: 3
Number of backtracks: 1746

[40, 35, 2]
Minimum number of time slots required: 3
Number of backtracks: 0

[11, 35, 5]
Can't be solved ¯\_(ツ)_/¯



## Experiment 2: Loosening the constraint

In [13]:
ratiosToTest = [
    [11, 200, 2],
    [11, 35, 5] # seeing if only scaling the # of startups the investors can rank
]

degreesOfConstraint = [
    [1, 0], # most constrained
    [0.9, 0.1],
    [0.8, 0.2],
    [0.7, 0.3],
    [0.6, 0.4],
    [0.5, 0.5] # least constrained
]

'''
Here, we've added a parameter to our graph-creating/edge-creating functions
that loosens or tightens the constraining-ness of investor's preferences.

[1,0], the default, means that no investor will have any conflicting
startups on their schedule (the tightest degree of constraint). The parameter must be an array of [p, 1-p].
'''

def cantBeScheduledTogether(startup1, startup2, investors, degreeOfConstraint=[1, 0]):
    for investor in investors:
        if (
            startup1['name'] != startup2['name'] and
            (startup1 in investor['rankedStartups'] and
            startup2 in investor['rankedStartups'])
            
        ):
            np.random.seed()
            return np.random.choice([True, False], p=degreeOfConstraint)
    return False

def generateStartupGraph(startups, investors, degreeOfConstraint=[1, 0]):
    startupGraph = OrderedDict()
    
    for startup in startups:
        neighbors = list(filter(lambda x: cantBeScheduledTogether(x, startup, investors, degreeOfConstraint), startups))
        neighborNames = list(map(lambda x: x['name'], neighbors))
        startupGraph[startup['name']] = neighborNames
    
    return startupGraph

for ratio in ratiosToTest:
    print(ratio)
    np.random.seed(123448)
    (startups, investors) = generateTestData(
        numberOfStartups=ratio[0],
        numberOfInvestors=ratio[1],
        numberOfRanksInvestorCanProvide=ratio[2]
    )
    
    for deg in degreesOfConstraint:
        print('Degree of constraint: ', deg)
        startupGraph = generateStartupGraph(startups, investors, degreeOfConstraint=deg)
        generateSchedule(startups, startupGraph, metrics=True)

[11, 200, 2]
Degree of constraint:  [1, 0]
Can't be solved ¯\_(ツ)_/¯

Degree of constraint:  [0.9, 0.1]
Minimum number of time slots required: 7
Number of backtracks: 0

Degree of constraint:  [0.8, 0.2]
Minimum number of time slots required: 6
Number of backtracks: 0

Degree of constraint:  [0.7, 0.3]
Minimum number of time slots required: 6
Number of backtracks: 0

Degree of constraint:  [0.6, 0.4]
Minimum number of time slots required: 5
Number of backtracks: 2

Degree of constraint:  [0.5, 0.5]
Minimum number of time slots required: 4
Number of backtracks: 0

[11, 35, 5]
Degree of constraint:  [1, 0]
Can't be solved ¯\_(ツ)_/¯

Degree of constraint:  [0.9, 0.1]
Minimum number of time slots required: 7
Number of backtracks: 0

Degree of constraint:  [0.8, 0.2]
Minimum number of time slots required: 6
Number of backtracks: 38

Degree of constraint:  [0.7, 0.3]
Minimum number of time slots required: 5
Number of backtracks: 224

Degree of constraint:  [0.6, 0.4]
Minimum number of time s